In [59]:
first_tools = [
        # Tool 1 - Get Exchange Rate
        { 
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the current exchange rate of a base currency and target currency",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "base_currency": {
                            "type": "string",
                            "description": "The base currency for exchange rate calculations, i.e. USD, EUR, RUB",
                        },
                        "target_currency": {
                            "type": "string", 
                            "description": "The target currency for exchange rate calculations, i.e. USD, EUR, RUB"
                        },
                        "date": {
                            "type": "string", 
                            "description": "A specific day to reference, in YYYY-MM-DD format."
                        },
                    },
                    "required": ["base_currency", "target_currency"],
                },
            },
        },
        # Tool 2 - Search Internet
        { 
            "type": "function",
            "function": {
                "name": "search_internet",
                "description": "Get internet search results for real time information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The query to search the web for",
                        }
                    },
                    "required": ["search_query"],
                },
            },
        },
        # Tool 3 - Get Weather
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the current weather for a specific location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {"type": "string"},
                        "unit": {"type": "string", "enum": ["metric", "imperial", "standard"]},
                    },
                    "required": ["location"],
                    "additionalProperties": False,
                },
            },
        }
    ]

In [96]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openweather_api_key = os.getenv("OPENWEATHER_API_KEY")

In [97]:
from openai import OpenAI
import json

client = OpenAI(api_key=api_key)

prompt = "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"

messages = [{"role": "user", 
             "content": prompt}]


response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=first_tools,
        tool_choice="auto",
    )

In [98]:
def pprint_response(response):
    print("--- Full Response ---\n")
    print(response, "\n")
    
    print("--- Chat Completion Message ---\n")
    print(response.choices[0].message, "\n")
    
    if response.choices[0].message.tool_calls:
        for i in range(0, len(response.choices[0].message.tool_calls)):
            print(f"--- Tool Call {i+1} ---\n")
            print(f"Function: {response.choices[0].message.tool_calls[i].function.name}\n")
            print(f"Arguments: {response.choices[0].message.tool_calls[i].function.arguments}\n")

In [99]:
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)

print(response.choices[0].message.tool_calls[2].function.name)
print(response.choices[0].message.tool_calls[2].function.arguments)

get_exchange_rate
{"base_currency": "USD", "target_currency": "JPY"}
search_internet
{"search_query": "current news in Peru"}


In [100]:
tool_calls = response.choices[0].message.tool_calls

tool_calls

[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'),
 ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='function')]

In [101]:
pprint_response(response)

--- Full Response ---

ChatCompletion(id='chatcmpl-AMb6g2rvwDcraGa2BIEV7lRJUVCHA', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type=

In [102]:
from duckduckgo_search import DDGS
import requests

In [103]:
def search_internet(search_query: str) -> list:
    results = DDGS().text(str(search_query), max_results=5)
    return results

def get_exchange_rate(base_currency: str, target_currency: str, date: str = "latest") -> float:
    
    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@{date}/v1/currencies/{base_currency.lower()}.json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get(base_currency.lower(), {}).get(target_currency.lower(), None)
    else:
        raise Exception(f"Failed to fetch exchange rate: {response.status_code}")
    
def get_weather(location: str, unit: str = "metric") -> dict:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&units={unit}&appid={openweather_api_key}"
    response = requests.get(url)
    return response.json()

In [104]:
search_internet("news in Peru")

[{'title': "Peru News | Today's Latest Stories | Reuters",
  'href': 'https://www.reuters.com/world/peru/',
  'body': "Peru's rebounding economy to grow 3.1% this year, cenbank forecasts. Peru's central bank expects that the Andean nation's economy will grow 3.1% this year and a further 3.0% in 2025, it said in a ..."},
 {'title': 'Peru declares state of emergency in parts of capital after crime spike ...',
  'href': 'https://www.reuters.com/world/americas/peru-declares-state-emergency-parts-capital-after-crime-spike-2023-09-18/',
  'body': "Peru's President Dina Boluarte speaks as she meets with foreign press, in Lima, Peru January 24, 2023. ... Reuters, the news and media division of Thomson Reuters, is the world's largest ..."},
 {'title': "What's behind the violence, protests in Peru? - NBC News",
  'href': 'https://www.nbcnews.com/news/latino/violence-protests-peru-rcna65543',
  'body': 'A wounded protester receives assistance at a demonstration in Cusco, Peru, on Wednesday. Ivan 

In [124]:
messages = [{"role": "user", "content": prompt}]

In [125]:
print("\nInitial Message: ", messages)


Initial Message:  [{'role': 'user', 'content': "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"}]


In [126]:
first_tools

[{'type': 'function',
  'function': {'name': 'get_exchange_rate',
   'description': 'Get the current exchange rate of a base currency and target currency',
   'parameters': {'type': 'object',
    'properties': {'base_currency': {'type': 'string',
      'description': 'The base currency for exchange rate calculations, i.e. USD, EUR, RUB'},
     'target_currency': {'type': 'string',
      'description': 'The target currency for exchange rate calculations, i.e. USD, EUR, RUB'},
     'date': {'type': 'string',
      'description': 'A specific day to reference, in YYYY-MM-DD format.'}},
    'required': ['base_currency', 'target_currency']}}},
 {'type': 'function',
  'function': {'name': 'search_internet',
   'description': 'Get internet search results for real time information',
   'parameters': {'type': 'object',
    'properties': {'search_query': {'type': 'string',
      'description': 'The query to search the web for'}},
    'required': ['search_query']}}},
 {'type': 'function',
  'funct

In [127]:
tool_calls

[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'),
 ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='function')]

In [128]:
response

ChatCompletion(id='chatcmpl-AMb6g2rvwDcraGa2BIEV7lRJUVCHA', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='function')]))], create

In [129]:
response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='function')])

In [130]:
functions = {
            "get_exchange_rate": get_exchange_rate,
            "search_internet": search_internet,
            "get_weather": get_weather,
        }

In [131]:
functions

{'get_exchange_rate': <function __main__.get_exchange_rate(base_currency: str, target_currency: str, date: str = 'latest') -> float>,
 'search_internet': <function __main__.search_internet(search_query: str) -> list>,
 'get_weather': <function __main__.get_weather(location: str, unit: str = 'metric') -> dict>}

In [132]:
messages

[{'role': 'user',
  'content': "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"}]

In [133]:
messages.append(response.choices[0].message)

In [134]:
messages

[{'role': 'user',
  'content': "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='functio

In [135]:
tool_calls = response.choices[0].message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'),
 ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'),
 ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='function')]

In [139]:
def ensure_string(content):
    if isinstance(content, (dict, list)):
        return json.dumps(content)
    elif isinstance(content, (int, float)):
        return str(content)
    elif isinstance(content, str):
        return content
    else:
        return str(content)

In [142]:
messages

[{'role': 'user',
  'content': "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='functio

In [143]:
function_to_call = []
function_args = []

for tool_call in tool_calls:
    function_to_call = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)

    print(f"\nCalling {function_to_call} with arguments {function_args}")

    chosen_function = eval(function_to_call)
    function_response = chosen_function(**function_args)

    print("\nFunction Response: ", function_response)

    tool_message = {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "function_name": function_to_call,
        "content": ensure_string(function_response),
    }
    
    print("\nNew Tool Message: ", tool_message)
    
    messages.append(tool_message)


Calling get_exchange_rate with arguments {'base_currency': 'USD', 'target_currency': 'JPY'}

Function Response:  151.90362698

New Tool Message:  {'tool_call_id': 'call_Fyhi4UU25uTBi0YrQU3xetSg', 'role': 'tool', 'function_name': 'get_exchange_rate', 'content': '151.90362698'}

Calling get_exchange_rate with arguments {'base_currency': 'USD', 'target_currency': 'TRY'}

Function Response:  34.31069893

New Tool Message:  {'tool_call_id': 'call_pzNhRCR4Truc2EE5ijYmHf0g', 'role': 'tool', 'function_name': 'get_exchange_rate', 'content': '34.31069893'}

Calling search_internet with arguments {'search_query': 'current news in Peru'}

Function Response:  [{'title': "Peru News | Today's Latest Stories | Reuters", 'href': 'https://www.reuters.com/world/peru/', 'body': 'Reuters.com is your online source for the latest Peru news stories and current events, ensuring our readers up to date with any breaking news developments'}, {'title': "Peru | Today's latest from Al Jazeera", 'href': 'https://www

In [144]:
print(len(messages))

messages

6


[{'role': 'user',
  'content': "How much is a dollar worth in Japan? How about Turkey? Whats the current news in Peru? What's the weather in Lima?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Fyhi4UU25uTBi0YrQU3xetSg', function=Function(arguments='{"base_currency": "USD", "target_currency": "JPY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_pzNhRCR4Truc2EE5ijYmHf0g', function=Function(arguments='{"base_currency": "USD", "target_currency": "TRY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_JcFyLF6KhzX3Bb1XpkCZ7Zw3', function=Function(arguments='{"search_query": "current news in Peru"}', name='search_internet'), type='function'), ChatCompletionMessageToolCall(id='call_YZEcsjZREkvXQsem99kp8def', function=Function(arguments='{"location": "Lima", "unit": "metric"}', name='get_weather'), type='functio

In [145]:
final_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )  

print("\nLLM Response: ", final_response)


LLM Response:  ChatCompletion(id='chatcmpl-AMbDyS88pZagVZP9YfDmWfBWhxGVd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of now, the exchange rates are as follows:\n\n- **Japan (JPY)**: 1 USD is approximately **151.90 JPY**.\n- **Turkey (TRY)**: 1 USD is approximately **34.31 TRY**.\n\n### Current News in Peru\n1. **[Latest News on Peru - Reuters](https://www.reuters.com/world/peru/)**: This source provides ongoing coverage of current events in Peru.\n2. **[Al Jazeera - Peru Updates](https://www.aljazeera.com/where/peru/)**: This platform has fact-based news and exclusive content on what's happening in Peru.\n3. **[NBC News - Violence and Protests in Peru](https://www.nbcnews.com/news/latino/violence-protests-peru-rcna65543)**: This article discusses recent violence and protests in Peru, related to political developments.\n4. **[State of Emergency Declared - Reuters](https://www.reuters.com/world/americas/peru-declares-state-em

In [147]:
print(final_response.choices[0].message.content)

As of now, the exchange rates are as follows:

- **Japan (JPY)**: 1 USD is approximately **151.90 JPY**.
- **Turkey (TRY)**: 1 USD is approximately **34.31 TRY**.

### Current News in Peru
1. **[Latest News on Peru - Reuters](https://www.reuters.com/world/peru/)**: This source provides ongoing coverage of current events in Peru.
2. **[Al Jazeera - Peru Updates](https://www.aljazeera.com/where/peru/)**: This platform has fact-based news and exclusive content on what's happening in Peru.
3. **[NBC News - Violence and Protests in Peru](https://www.nbcnews.com/news/latino/violence-protests-peru-rcna65543)**: This article discusses recent violence and protests in Peru, related to political developments.
4. **[State of Emergency Declared - Reuters](https://www.reuters.com/world/americas/peru-declares-state-emergency-parts-capital-after-crime-spike-2023-09-18/)**: Peru has declared a state of emergency in parts of the capital due to a spike in crime.
5. **[BBC News on Peru](https://www.bbc.co